# Process and barcode sequencing reads
This script takes paired read (read 1: barcode + UMI (27bp), read 2: staggers + UMI + partial barcode (49bp)) fastq files and does the following:
1. trims read 2 adapter sequences to recover barcode+UMI sequence
2. merges read 1 and 2 with FLASH
3. identifies and counts barcodes

In [1]:
import glob
import os
import subprocess
import regex
import gzip
from Bio import SeqIO
import pandas as pd
import multiprocessing as mp
import numpy as np

# check number of available cores
len(os.sched_getaffinity(0))

20

## Inspect fastq files for quality control using FastQC

In [2]:
!mkdir ~/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/
!fastqc -q -o ~/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/ ~/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/*fastq.gz 

mkdir: cannot create directory ‘/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/’: File exists


## Summarize FastQC output with MultiQC

In [3]:
!multiqc -o ~/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/ ~/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/

[WARNING]         multiqc : MultiQC Version v1.11 now available!
[INFO   ]         multiqc : This is MultiQC v1.9
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc
Searching 145 files..  [####################################]  100%          
[INFO   ]          fastqc : Found 72 reports
[INFO   ]         multiqc : Compressing plot data
[WARNING]         multiqc : Previous MultiQC output found! Adjusting filenames..
[WARNING]         multiqc : Use -f or --force to overwrite existing reports instead
[INFO   ]         multiqc : Report      : ../../../scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/multiqc_report_1.html
[INFO   ]         multiqc : Data        : ../../../scratch/yeast/crispey3/pool1_scm_aug2021/fastq/fastqc/multiqc_data_1
[INFO   ]         multiqc : MultiQC complete


## Map fastq file names to sample names

In [4]:
# key to map fastq names to output names
seqID_to_sampleName = {}
sample_key_file = "/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/SampleKey-18146-36.txt"
with open(sample_key_file, 'r') as sample_key:
    sample_key.readline() # skip header
    for line in sample_key:
        seqID, sampleName = line.rstrip().split("\t")
        sampleName = sampleName.replace("_","-")
        seqID_to_sampleName[seqID] = sampleName


## Trim read 2 adapters with cutadapt
Remove staggers, leaving the UMI+partial barcode sequence (19-26bp, may be shorter depending on quality trimming)

In [5]:
# working directory with fastq files
working_dir="/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/"
os.chdir(working_dir)

# get read 2 files for trimming
fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R2_001.fastq.gz")])

In [6]:
# cutadapt parameters to trim read 2 to get barcode+UMI (27bp)
adapter_5prime = 'GGCCAGTTTAAACTT'

num_of_cores = len(os.sched_getaffinity(0))
err = 0.2 # fraction tolerated for adapter matching
min_r2_length = 12 # R2 must contain at least UMI sequence and some of SphI linker
output_dir_name = 'trimmed'

In [7]:
# store sample key in regex pattern
pattern = regex.compile('|'.join(seqID_to_sampleName.keys()))

# trim read 2, filter untrimmed read pairs
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # rename output files by sample key stored in seqID_to_sampleName 
    fastq_file = os.path.basename(fastq_path)
    output_file_r2 = pattern.sub(lambda x: seqID_to_sampleName[x.group()], fastq_file).replace("_001.fastq.gz", "_001_trimmed.fastq.gz")
    output_file_r1 = output_file_r2.replace("_R2_", "_R1_")

    print('Trimming: ' + fastq_path)
    
    cutadapt_cmd = ["cutadapt", "-g", adapter_5prime, 
                    "-j", str(num_of_cores), 
                    "-e", str(err), 
                    #"-q", "20", # use -q for miseq/hiseq quality trimming
                    "--nextseq-trim", "20", # use this option for nextseq quality trimming
                    "--discard-untrimmed", "-m", str(min_r2_length), 
                    "--pair-filter=first", 
                    "-o", output_dir+output_file_r2, "-p", output_dir+output_file_r1,
                    fastq_path, fastq_path.replace("L001_R2_", "L001_R1_")]
    
    subprocess.run(cutadapt_cmd)

    print("Output files:")
    print(output_file_r2)
    print(output_file_r1)
    print()
    
print('Done trimming!')

Trimming: /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/18146-36-01V2-01_S1_L001_R2_001.fastq.gz
Output files:
P1-SCM-T1-F1_S1_L001_R2_001_trimmed.fastq.gz
P1-SCM-T1-F1_S1_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/18146-36-01V2-02_S2_L001_R2_001.fastq.gz
Output files:
P1-SCM-T2-F1_S2_L001_R2_001_trimmed.fastq.gz
P1-SCM-T2-F1_S2_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/18146-36-01V2-03_S3_L001_R2_001.fastq.gz
Output files:
P1-SCM-T3-F1_S3_L001_R2_001_trimmed.fastq.gz
P1-SCM-T3-F1_S3_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/18146-36-01V2-04_S4_L001_R2_001.fastq.gz
Output files:
P1-SCM-T4-F1_S4_L001_R2_001_trimmed.fastq.gz
P1-SCM-T4-F1_S4_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/18146-36-01V2-05_S5_L001_R2_001.fastq.gz
Output files:
P1-SCM-T5-F1_S5_

## Merge read 1 and read 2 with FLASH to produce final barcode+UMI sequence

In [8]:
working_dir="/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/"
os.chdir(working_dir)

fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R1_001_trimmed.fastq.gz")])

In [9]:
# FLASH parameters
min_overlap = 12 # min overlap cannot be longer than the shorter read.
max_overlap = 27
max_mismatch = 0.4 #default is 0.25, set 0.4 if seq quality is low
output_dir_name = 'merged'

In [10]:
# use FLASH to merge trimmed-filtered read 2 and read 1 data to produce final 27bp sequence containing barcode and UMI data
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # check output file naming 
    output_prefix = os.path.basename(fastq_path).split("_")[0]+"_barcode"
    print('Merging', fastq_path, 'and', fastq_path.replace("_R1_", "_R2_"))
    
    flash_cmd = ["flash", "-m", str(min_overlap), "-M", str(max_overlap),
                 "-x", str(max_mismatch), #"-O", # use -O if innie-only merging does not work
                 "-o", output_prefix, "-d", output_dir, 
                 "--compress", 
                 fastq_path, fastq_path.replace("_R1_", "_R2_")]
    subprocess.run(flash_cmd)
    print(output_prefix, "merged")

print('Done merging!')

Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F1_S1_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F1_S1_L001_R2_001_trimmed.fastq.gz
P1-SCM-T1-F1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F2_S7_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F2_S7_L001_R2_001_trimmed.fastq.gz
P1-SCM-T1-F2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F3_S13_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F3_S13_L001_R2_001_trimmed.fastq.gz
P1-SCM-T1-F3_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F4_S19_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T1-F4_S

P1-SCM-T6-F1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F2_S12_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F2_S12_L001_R2_001_trimmed.fastq.gz
P1-SCM-T6-F2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F3_S18_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F3_S18_L001_R2_001_trimmed.fastq.gz
P1-SCM-T6-F3_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F4_S24_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F4_S24_L001_R2_001_trimmed.fastq.gz
P1-SCM-T6-F4_barcode merged
Merging /scratch/users/rang/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/P1-SCM-T6-F5_S30_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/pool1_scm_aug2

## (optional) Downsample reads for analysis
Use seqtk in command line to downsample fastq files prior to assembling counts matrix.<br>
e.g. seqtk sample -s100 read1.fq 10000 > sub1.fq

## Count barcodes
Counting barcodes consists of several steps. First, parse each fastq file and count all sequences. After assembling into an initial sequences counts matrix, extract the barcode and UMI sequences and map them to a reference table of barcodes and UMIs. Counts for ID-able sequences are onsolidated into a final counts matrix for input to DESeq2

In [11]:
def count_seqs(fastq_file, min_seq_length, max_seq_length):
    '''
    Parses a fastq file and counts sequences. Returns dict of counts
    '''
    seq_counts_dict = {}
    # parse fastq
    with gzip.open(fastq_file, 'rt') as fastq:
        for read in SeqIO.parse(fastq, "fastq"):
            # filter for sequences within min/max length and contains no N's
            if min_seq_length <= len(read.seq) <= max_seq_length and read.seq.count("N")==0:
                sequence = str(read.seq)
                # count sequence
                try:
                    seq_counts_dict[sequence] += 1
                except KeyError:
                    seq_counts_dict[sequence] = 1
    
    return seq_counts_dict

    
def map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq):
    '''
    splits a sequence into barcode and UMI, maps to barcode table and UMI list to assign ID
    does NOT do UMI mapping if umi_list is empty.
    '''
    final_id = 'UNKNOWN'
    
    barcode, umi = split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq)
    
    if len(barcode)<barcode_length/2 or len(umi)<umi_length/2:
        # Barcode/UMI too short
        return final_id

    # assign barcode ID
    barcode_id = assign_barcode(barcode=barcode, barcode_table=barcode_table, error=1)
    if barcode_id == 'UNKNOWN':
        # Barcode cannot be identified
        return barcode_id
    else:
        # add barcode ID to final ID
        final_id = barcode_id

    # assign UMI ID (if applicable)
    if len(umi_list)>0:
        umi_id = assign_umi(umi=umi, umi_list=umi_list, error=1)
        if umi_id == 'UNKNOWN':
            # UMI cannot be identified
            return umi_id
        else:
            # add UMI ID to final ID
            final_id = '-'.join([final_id, str(umi_id)])
    
    return final_id
    

def split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq):
    '''
    Splits seq by linker_seq and returns barcode and UMI sequence
    If linker seq cannot be found (e.g. sequencing error) or yields multiple splits,
    fall back to splitting by base position.
    '''
    barcode = ''
    umi = ''
    
    try:
        # split by linker
        barcode, umi = seq.split(linker_seq) # can try error tolerant regex?
    except ValueError:
        # split by base position
        if barcode_length>0 and umi_length==0:
            barcode = seq[:barcode_length]
        else:
            umi = seq[-umi_length:]
            barcode = seq[:-(umi_length+len(linker_seq))] # may return partial barcodes for short sequences

    return (barcode, umi)


def assign_barcode(barcode, barcode_table, error):
    '''
    Searches barcode table for barcode sequence and returns unique barcode ID
    Tries perfect match first, then error-tolerant regex
    '''
    if barcode == '':
        barcode_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            barcode_id = barcode_table.loc[barcode, 'Unique_ID']
        except KeyError:
            # search by error-tolerant regex
            pattern = "(?:"+barcode+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in barcode_table.index]
            if sum(search)==1:
                barcode_id = barcode_table.loc[search, 'Unique_ID'][0]
            else:
                # barcode cannot be identified
                barcode_id = 'UNKNOWN'
    
    return barcode_id


def assign_umi(umi, umi_list, error):
    '''
    Searches umi list for umi sequence and returns 1-index position as umi ID
    Tries perfect match first, then error-tolerant regex
    '''
    if umi == '':
        umi_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            umi_id = umi_list.index(umi)+1
        except ValueError:
            # search by error-tolerant regex
            pattern = "(?:"+umi+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in umi_list]
            if sum(search) == 1:
                umi_id = search.index(True)+1
            else:
                # UMI cannot be identified
                umi_id = 'UNKNOWN'
    
    return umi_id


In [12]:
working_dir="/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/fastq/trimmed/merged/"
os.chdir(working_dir)

# merged reads to count barcodes from
fastq_list = sorted(glob.glob("*extendedFrags.fastq.gz")) # check for file name
# sample names for each fastq
sample_name_list = [fastq_file.split("_")[0] for fastq_file in fastq_list] # adjust accordingly to generate sample name for counts matrix

#output directory
output_dir = "/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/counts/"

# sequence counts file (before combining)
seq_counts_filename = "seq_counts.txt"

# mapped barcode-UMI counts file
barcode_counts_filename = "barcode_counts.txt"


# open barcode reference file
barcode_reference_file = '/home/users/rang/crispey3/library_design/Input/12BP_PBCs_well_grouped.csv'
barcode_table = pd.read_csv(barcode_reference_file, index_col=1)

# approved list of UMIs used in cloning CRISPEY3 plasmid
umi_list = ['ACGCGTGAA',
            'ATGTGGCTC',
            'CAGAGGATC',
            'CTGTGGCAA',
            'GTGTGATTC',
            'TAGAGGACT',] # only first 6 UMIs were included in cloning CRISPEY3 libaries
#             'AAGAGCCTC',
#             'AAGAGGAGG',
#             'ATGTGCGAA',
#             'ATGTGTAGG',
#             'CAGAGCCAA',
#             'CTGTGATGG',
#             'CTGTGTATC',
#             'GAGAGGAAA',
#             'TCGCGGTAA',
#             'TTGTGCGTC']
umi_list = sorted(umi_list)


In [13]:
# count sequences in each fastq file
fastq_dict = dict(zip(sample_name_list, fastq_list))
with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
    seq_counts_df = {sample_name : pool.apply_async(count_seqs, (fastq_file, 20, 27)) for sample_name, fastq_file in fastq_dict.items()}
    seq_counts_df = {sample_name : res.get() for sample_name, res in seq_counts_df.items()}
    
# merge to dataframe
seq_counts_df = pd.DataFrame.from_dict(seq_counts_df, orient="columns")
seq_counts_df.index.name = 'sequence'

# write to file to inspect
os.makedirs(output_dir, exist_ok=True)
seq_counts_df.to_csv(output_dir+seq_counts_filename, sep="\t")


In [13]:
# seq_counts_df = pd.read_csv(output_dir+seq_counts_filename, sep="\t", index_col='sequence')

In [14]:
# map each sequence in seq_counts_df to barcode-UMI ID
mapped_counts_df = seq_counts_df.copy()
mapped_counts_df = mapped_counts_df.reset_index()

# filter out low count barcodes (computationally expensive to map these rare barcodes, minimal impact to total count (~2%))
cutoff = 3
mapped_counts_df = mapped_counts_df.loc[mapped_counts_df.sum(axis=1)>=cutoff]

# do multiprocessing for sequence mapping
num_of_cores = len(os.sched_getaffinity(0))
with mp.Pool(num_of_cores) as pool:
    barcode_umi_id_list = [pool.apply_async(map_seq_to_barcode_umi, (seq, barcode_table, umi_list, 12, 9, 'GCATGC')) for seq in mapped_counts_df['sequence']]
    barcode_umi_id_list = [res.get() for res in barcode_umi_id_list]
    
# mapped_counts_df['barcode_umi_id'] = [x for res in barcode_umi_id_lists for x in res] #pd.concat(barcode_umi_id_lists)
mapped_counts_df['barcode_umi_id'] = barcode_umi_id_list
display(mapped_counts_df)

# remove unknowns
mapped_counts_df = mapped_counts_df.query('barcode_umi_id!="UNKNOWN"')

# consolidate counts
mapped_counts_df = mapped_counts_df.groupby('barcode_umi_id').sum().fillna(0).astype(int)
mapped_counts_df.index.name = 'barcode'
display(mapped_counts_df)

# write all counts to output file
os.makedirs(output_dir, exist_ok=True)
mapped_counts_df.to_csv(output_dir+barcode_counts_filename, sep="\t")


# # one-liner to map sequences to barcode-umi IDs
# # warning: single-threaded, expected to be slow
# mapped_counts_df['barcode_umi_id'] = mapped_counts_df['sequence'].apply(map_seq_to_barcode_umi, args=(barcode_table, umi_list, 12, 9, 'GCATGC'))


,sequence,P1-SCM-T1-F1,P1-SCM-T1-F2,P1-SCM-T1-F3,P1-SCM-T1-F4,P1-SCM-T1-F5,P1-SCM-T1-F6,P1-SCM-T2-F1,P1-SCM-T2-F2,P1-SCM-T2-F3,...,P1-SCM-T5-F4,P1-SCM-T5-F5,P1-SCM-T5-F6,P1-SCM-T6-F1,P1-SCM-T6-F2,P1-SCM-T6-F3,P1-SCM-T6-F4,P1-SCM-T6-F5,P1-SCM-T6-F6,barcode_umi_id
0,ATGTCTAGCAATGCATGCGTGTGATTC,512.0,418.0,447.0,530.0,423.0,886.0,505.0,584.0,298.0,...,428.0,479.0,686.0,394.0,545.0,528.0,607.0,531.0,1002.0,015_034-5
1,TGCGACTGTTGAGCATGCACGCGTGAA,657.0,525.0,655.0,663.0,542.0,1226.0,644.0,845.0,467.0,...,527.0,680.0,978.0,574.0,663.0,660.0,574.0,721.0,1221.0,034_029-1
2,AAGATCAATCAGGCATGCCTGTGGCAA,203.0,177.0,244.0,231.0,94.0,401.0,218.0,180.0,108.0,...,162.0,172.0,194.0,186.0,117.0,184.0,117.0,230.0,402.0,030_035-4
3,GTGATACTTCCGGCATGCCTGTGGCAA,49.0,48.0,48.0,48.0,28.0,94.0,50.0,62.0,35.0,...,34.0,46.0,73.0,40.0,74.0,69.0,40.0,59.0,84.0,033_074-4
4,TGAAGTCCTTTAGCATGCCAGAGGATC,12581.0,10359.0,13820.0,13810.0,7195.0,24012.0,13603.0,12516.0,6920.0,...,9240.0,10853.0,16355.0,11782.0,9125.0,11071.0,7307.0,14002.0,25682.0,Ladder_040-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387244,AGAGTCCGCCTTGCATGCCTGTGTCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,011_091-4
1387253,TACGGACTATCCGCATGCCTGTGGCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,192_058-4
1387600,CCTCGGATAAGCTCATTCACGCGTGAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,014_053-1
1391567,TCTGCGTTATAAGCATGCCTGTGGCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,004_048-4


,P1-SCM-T1-F1,P1-SCM-T1-F2,P1-SCM-T1-F3,P1-SCM-T1-F4,P1-SCM-T1-F5,P1-SCM-T1-F6,P1-SCM-T2-F1,P1-SCM-T2-F2,P1-SCM-T2-F3,P1-SCM-T2-F4,...,P1-SCM-T5-F3,P1-SCM-T5-F4,P1-SCM-T5-F5,P1-SCM-T5-F6,P1-SCM-T6-F1,P1-SCM-T6-F2,P1-SCM-T6-F3,P1-SCM-T6-F4,P1-SCM-T6-F5,P1-SCM-T6-F6
barcode,,,,,,,,,,,,,,,,,,,,,
001_001-1,8,10,7,8,3,9,9,5,1,4,...,10,3,4,1,5,8,4,11,9,1
001_001-2,14,9,13,16,20,26,16,21,10,16,...,14,11,20,14,11,9,13,17,16,24
001_001-3,11,10,3,13,8,28,15,17,11,7,...,6,8,7,12,11,17,0,12,7,9
001_001-4,5,8,2,11,8,21,6,11,4,3,...,13,4,4,8,10,7,5,4,7,21
001_001-5,6,2,4,3,5,10,6,2,3,7,...,9,2,5,11,2,7,4,5,10,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ladder_043-2,126,132,169,162,151,336,184,208,130,131,...,252,165,204,277,176,196,170,181,203,329
Ladder_043-3,267,218,291,266,243,544,264,295,189,201,...,391,197,287,382,311,268,305,226,370,500
Ladder_043-4,290,223,325,320,174,531,292,288,181,212,...,367,218,282,458,288,242,242,173,352,683


## (optional) Combine counts across UMIs per barcode
The counts of different UMIs of the same barcode can be added together to produce a stacked counts matrix

In [15]:
# combine counts from different UMIs of the same barcode
stacked_counts_filename = "stacked_barcode_counts.txt"

stacked_counts_df = mapped_counts_df.groupby(by=lambda x: x.split('-')[0]).sum()
stacked_counts_df.to_csv(output_dir+stacked_counts_filename, sep="\t")

In [2]:
# # adapt ladder fastq files by trimming to new seq format (R1: 27bp, R2: 49bp)
# r1_len = 27
# r2_len = 49

# # working directory with fastq files
# working_dir="/home/users/rang/scratch/yeast/tmp/ladder_pilot_adapted/fastq/"
# os.chdir(working_dir)

# # trim R1
# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R1_001.fastq.gz")])
# for fastq_path in fastq_list:
#     output_path = fastq_path.replace("_R1_001", "_R1_adapted")
#     cutadapt_cmd = ["cutadapt", "-l", str(r1_len),
#                     "-o", output_path, fastq_path]
#     subprocess.run(cutadapt_cmd)
    
# # trim R2
# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R2_001.fastq.gz")])
# for fastq_path in fastq_list:
#     output_path = fastq_path.replace("_R2_001", "_R2_adapted")
#     cutadapt_cmd = ["cutadapt", "-l", str(r2_len),
#                     "-o", output_path, fastq_path]
#     subprocess.run(cutadapt_cmd)

In [ ]:
# # one-step count_barcodes function
# # warning: less efficient since barcode-UMI mapping is done per sample, rather than a single time after all sequences are counted

# def count_barcodes(fastq_file, barcode_table, umi_list, 
#                    min_seq_length=20, barcode_length=12, umi_length=9, linker_seq='GCATGC'):
#     '''
#     parses fastq file to count sequences, then extracts barcode-UMI info from sequences and assigns ID based
#     on provided reference barcode_table and umi_list. Finally, consolidates counts by assigned ID and returns
#     a dict of barcode counts.
#     does NOT do UMI mapping if umi_list is empty.
#     '''
#     # set max_seq_length
#     max_seq_length = barcode_length+umi_length+len(linker_seq) # this could be adjusted to allow insertions
#     # alphabetical sort umi_list
#     umi_list = sorted(umi_list)
    
#     # count raw sequences
#     seq_counts = count_seqs(fastq_file, min_seq_length, max_seq_length)
    
#     # consolidate barcode counts
#     barcode_counts_dict = {}
#     for seq, count in seq_counts.items():
#         # assign barcode-UMI ID
#         assigned_id = map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq)
#         if assigned_id:
#             try:
#                 barcode_counts_dict[assigned_id] += count
#             except KeyError:
#                 barcode_counts_dict[assigned_id] = count
    
#     return barcode_counts_dict


# fastq_dict = dict(zip(sample_name_list, fastq_list))
# with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
#     all_counts_df = {sample_name : pool.apply_async(count_barcodes, (fastq_file, barcode_table, umi_list)) for sample_name, fastq_file in fastq_dict.items()}
#     all_counts_df = {sample_name : res.get() for sample_name, res in all_counts_df.items()}
    
# # write all counts to output file
# os.makedirs(output_dir, exist_ok=True)
# all_counts_df = pd.DataFrame.from_dict(all_counts_df, orient="columns")
# all_counts_df.index.name = 'barcode'
# all_counts_df.to_csv(output_dir+counts_filename, sep="\t")